__This script enables one to extract connectivity information from the NeuPrint API (https://neuprint.janelia.org/) for the purpose of creating a gexf network file. The gexf format allows each node to possess additional label information that might include its voxel size, brain region ROIs (regions of interest), net connectivity weight, etc. Once the gexf file has been generated, it can be viewed and manipulated using the robust tools of the program Gephi (https://gephi.org/)__ 

___Inputs___ <br>
_FIBSEM ID LIST_: list type; the list should consist of hemibrain Body IDs (from NeuPrint), seperated by commas <br>
_Min Voxels_: integer type; if the network consists of fragments that border the edge of the volume, one might want to remove all Body IDs below a certain voxel threshold.  <br>
_Min Weight_: integer type; remove any connections below a certain weight threshold  <br>
_dataset_: string type; choose which of the three versions of the hemibrain dataset you'd like to extract the information from ('hemibrain:v1.0.1', 'hemibrain:v1.1', 'hemibrain:v1.2') <br>
_token ID_: string type; the authentication token can be found on the NeuPrint page under the user profile <br>


___Output___
_gefx file_: this network file can be imported to the program Gephi, to conduct further analysis and manipulation

In [1]:
def NeuPrint_Multigraph(FIBSEM_ID_LIST, Min_Voxels, Min_Weight, dataset, token_ID):#Recommend Min_Voxels=16000000, and Min_Weight=3 
    import networkx as nx
    from neuprint import Client
    from neuprint import fetch_adjacencies
    from neuprint import fetch_neurons
    
    #establish Client
    c = Client('neuprint.janelia.org', dataset=str(dataset), token=str(token_ID))
    print(c.fetch_version())
    
    InputOutputList = []
    G = nx.MultiDiGraph()
    
    
    for Neuron in FIBSEM_ID_LIST:
        try:
            print(Neuron)
            #import input and output dataframes from NeuPrint
            neuron_dfin, inputs_df = fetch_adjacencies(None, [Neuron])
            neuron_dfout, outputs_df = fetch_adjacencies([Neuron],None)
            print("Done!")
            #create graph and FIBSEM_ID node
            G.add_node(Neuron)
            #add size attribute to FIBSEM_ID node 
            neuron_df, roi_counts_df = fetch_neurons(Neuron)
            G.node[Neuron]['size'] = int(neuron_df['size'])
            #add neuropil attribute to node
            G.node[Neuron]['roi'] = str(roi_counts_df['roi'].max())
            #add input nodes to G and create edges with original bodyID
            for index, r1 in inputs_df.iterrows():
                fetch_temp, roi_counts_df = fetch_neurons(r1['bodyId_pre'])
                #skip ID if below size minimum
                if int(fetch_temp['size']) < Min_Voxels:
                    continue
                #skip ID if orphan
                A=fetch_temp["status"].to_string()
                if A[5:11] == 'Orphan':
                    continue
                #skip ID if below minimum connection weight
                if int(r1['weight']) < Min_Weight:
                    continue
                #add ID node to graph
                G.add_node(r1['bodyId_pre'])
                #add size attribute to node
                G.node[r1['bodyId_pre']]['size'] = int(fetch_temp['size'])
                #add neuropil attribute to node
                G.node[r1['bodyId_pre']]['roi'] = str(roi_counts_df['roi'].max())
                #add edges to graph
                for n in range(int(r1['weight'])):
                    G.add_edge(str(r1['bodyId_pre']),str(Neuron)) 
                #add bodyID to input/output list
                InputOutputList.append(r1['bodyId_pre']) 
            #add output nodes to G and create edges with original bodyID                     
            for index, r1 in outputs_df.iterrows():
                fetch_temp, roi_counts_df = fetch_neurons(r1['bodyId_post'])
                #skip ID if below size minimum
                if int(fetch_temp['size']) < Min_Voxels:
                    continue
                #skip ID if orphan
                A=fetch_temp["status"].to_string()
                if A[5:11] == 'Orphan':
                    continue
                #skip ID if below minimum connection weight
                if int(r1['weight']) < Min_Weight:
                    continue
                #add ID node to graph
                G.add_node(r1['bodyId_post'])
                #add size attribute to node
                G.node[r1['bodyId_post']]['size'] = int(fetch_temp['size'])
                #add neuropil attribute to node
                G.node[r1['bodyId_post']]['roi'] = str(roi_counts_df['roi'].max())
                #add edges to graph
                for n in range(int(r1['weight'])):
                    G.add_edge(str(Neuron),str(r1['bodyId_post']))
                #add bodyID to input/output list
                InputOutputList.append(r1['bodyId_post'])
        except:
            pass
        
    #remove duplicates from input/output list
    InputOutputList1 = list( dict.fromkeys(InputOutputList) )
    #remove original IDs from input/output list
    InputOutputList2 = [i for i in InputOutputList1 if i not in FIBSEM_ID_LIST]
    
    #add edges between inputs/outputs of original bodyIDs
    for id1 in InputOutputList2:
        try:
            fetch_temp, temp_inputs_df = fetch_adjacencies(None, [id1])
            for index, r1 in temp_inputs_df.iterrows():
                if int(r1['weight']) < Min_Weight:
                        continue
                for id2 in InputOutputList2:
                    if r1['bodyId_pre'] == id2:
                        for r in range(int(r1['weight'])):
                            G.add_edge(str(id2),str(id1))
        #if there's an error in creating the bodyID dataframe
        except:
            print("The connectivity dataframe of the bodyID",id1,"encountered an error.")
            pass
        
    #remove edges between original ID list
    for ID1 in FIBSEM_ID_LIST:
        for ID2 in FIBSEM_ID_LIST:
            try:
                E = G.number_of_edges(str(ID1),str(ID2))
                for i in range(E):
                    G.remove_edge(str(ID1),str(ID2))
            except:
                pass
            
    #add edges back between original ID list
    for ID1 in FIBSEM_ID_LIST:
        for ID2 in FIBSEM_ID_LIST:
            try:
                fetch_temp, inputs_df = fetch_adjacencies([ID1],[ID2])
                E = sum(inputs_df['weight'])
                #skip ID if below minimum connection weight
                if int(E) < Min_Weight:
                    continue
                for i in range(E):
                    G.add_edge(str(ID1),str(ID2))
               
            #if there's an error in creating the bodyID dataframe
            except:
                pass  

        
    #write gefx file for use in Gephi
    nx.write_gexf(G, 'Atest'+'.gexf')